In [1]:
import sys
sys.path.append("..")

#import jax
#jax.config.update("jax_enable_x64", True)

from utils import *
from model_tn import *
from keras_utils import *
from jax_utils import *

keras.mixed_precision.set_global_policy("mixed_float16")
#keras.mixed_precision.set_global_policy("float32")
#keras.mixed_precision.set_global_policy("float64")

In [2]:
class TSCANToEnd(keras.Model):
    
    def __init__(self, model):
        super().__init__()
        self.inner = model
    
    def call(self, x, training=None):
        x_ = x[:, 1:] - x[:, :-1]
        x_ = (x_ - ops.mean(x_, axis=(2,3 ), keepdims=True))/(ops.std(x_, axis=(2, 3), keepdims=True)+1e-6)
        return self.inner((ops.concatenate([x_, x_[:, -1:]], axis=1), ops.mean(x, axis=(1, ), keepdims=True)), training=training)

model = TSCANToEnd(TSCAN())
model.compile(optimizer='adamw', loss='mse')
model(np.random.random((4, 160, 36, 36, 3)));
model.inner.summary()

Model: "tscan"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (640, 34, 34, 32)      │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (640, 15, 15, 64)      │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_2 (Sequential)       │ (4, 34, 34, 32)        │        10,144 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_3 (Sequential)       │ (4, 15, 15, 64)        │        55,424 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_4 (Sequential)       │ (4, 34, 34, 1)         │            33 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_5 (Sequential)       │ (4, 15, 15, 1)         │            65 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_6 (Sequential)       │ (640, 17, 17, 32)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_7 (Sequential)       │ (640, 1)               │       401,665 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 532,899 (2.03 MB)

 Trainable params: 532,899 (2.03 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
@jax.jit
@partial(shard_map, mesh=mesh, in_specs=P('x'), out_specs=P('x'))
def diff_label(x, y):
    y = ops.concatenate([y[:,1:]-y[:,:-1], ops.zeros_like(y[:,:1])], axis=1)
    return x, y

batch_size = 128

tape = "/root/ssd_cache/rppg_training_data/mmpd_160x36x36_all"

train_tape = load_datatape(tape, fold='train', batch=batch_size)
valid_tape = load_datatape(tape, fold='val', extended_hr='False', batch=batch_size)

#train_rlap = DatatapeMonitor(train_rlap)
training_set, validation_set = KerasDataset(train_tape), KerasDataset(valid_tape)

training_set = training_set.apply_fn(compress_aug)
training_set = training_set.apply_fn(diff_label)
validation_set = validation_set.apply_fn(diff_label)

mmpd_160x36x36_all             datatape has been loaded.     7552 items total. fold=train selected
mmpd_160x36x36_all             datatape has been loaded.     1664 items total. fold=val&extended_hr=False selected


In [4]:
#model.fit(training_set, validation_data=validation_set, epochs=10, callbacks=[keras.callbacks.ModelCheckpoint('../weights/tscan_mmpd.weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)])
train(model, training_set, validation_set, epochs=20, check_point_path='../weights/tscan_mmpd.weights.h5')
model.load_weights('../weights/tscan_mmpd.weights.h5')

Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch   1:  Training loss:0.1241	Validation loss:0.1163	


Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

...

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch  19:  Training loss:0.1169	Validation loss: 0.114	


Training:   0%|          | 0/59 [00:00<?, ?it/s]

Validating:   0%|          | 0/13 [00:00<?, ?it/s]

Epoch  20:  Training loss:0.1167	Validation loss: 0.114	


In [5]:
eval_on_dataset(dataset_H5_rlap, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_MMPD_RLAP.h5', cumsum=True, scenes=['R1', 'R2', 'R3', 'R4'])
get_metrics('../results/TSCAN_MMPD_RLAP.h5', dropped='False')

  0%|          | 0/220 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,11.597±0.193,17.951±3.135,15.333±0.274,0.20103
Whole,7.154±0.463,9.918±3.457,9.288±0.602,0.46097


In [6]:
eval_on_dataset(dataset_H5_cohface, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_MMPD_COHFACE.h5', cumsum=True, fps=30)
get_metrics('../results/TSCAN_MMPD_COHFACE.h5')

  0%|          | 0/164 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,13.199±0.368,20.54±4.54,19.884±0.597,0.12147
Whole,9.34±0.686,12.818±4.899,14.52±1.194,0.28744


In [7]:
eval_on_dataset(dataset_H5_pure, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_MMPD_PURE.h5', cumsum=True)
get_metrics('../results/TSCAN_MMPD_PURE.h5')

  0%|          | 0/59 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,15.15±0.73,25.213±7.229,21.371±0.943,0.25303
Whole,12.879±2.063,20.42±11.414,18.113±2.229,0.45387


In [8]:
eval_on_dataset(dataset_H5_ubfc_rppg2, pmodel(model), 160, (36, 36), step=4, batch=16, save='../results/TSCAN_MMPD_UBFC.h5', cumsum=True)
get_metrics('../results/TSCAN_MMPD_UBFC.h5')

  0%|          | 0/42 [00:00<?, ?it/s]

,MAE,RMSE,MAPE,R
Window,10.151±0.725,19.412±6.26,10.274±0.702,0.6047
Whole,8.764±1.472,12.952±6.721,8.712±1.426,0.79557
